In [12]:
import os
import numpy as np
import cv2


CAM_ENUM=1
CAM_WIDTH=640
CAM_HEIGHT=480
SAMPLE_SIZE=30

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.\
objpoints = [] # 3d point in real world spaceq
imgpoints = [] # 2d points in image plane.


camera = cv2.VideoCapture(CAM_ENUM, cv2.CAP_DSHOW)

camera.set(cv2.CAP_PROP_FRAME_WIDTH, CAM_WIDTH)  # Set the width
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, CAM_HEIGHT)  # Set the height

while True:
    successful, img = camera.read()
    if successful:
        img = cv2.flip(img,1)

        cv2.imshow("Webcam", img) # This will open an independent window
            

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, (6,7),None)
        # If found, add object points, image points (after refining them)
        if ret == True:           
            corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
            
           
            # Draw and display the corners
            cv2.drawChessboardCorners(img, (7,6), corners2, ret)
            cv2.imshow('Webcam', img)
            objpoints.append(objp)
            imgpoints.append(corners2)
            


    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        cv2.destroyAllWindows()
        camera.release()
        break

tempobjpoints = []
tempimgpoints = []
index = 0
for i in range(SAMPLE_SIZE):
    tempobjpoints.append(objpoints[index])
    tempimgpoints.append(imgpoints[index])
    index += int ((len(objpoints)-1) / SAMPLE_SIZE)
    
    
objpoints = tempobjpoints
imgpoints  = tempimgpoints

Calibration calculation     

In [13]:
img = gray
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
cv2.imshow('Un-calibrated image',img)

h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]

cv2.imshow('Calibrated image', dst)


while True:
    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        cv2.destroyAllWindows()
        camera.release()
        break
    
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )


total error: 4.798880096752648


In [14]:
import os

fileNum=0
for filename in os.listdir("./"):
        if filename.endswith('.npz'):
            fileNum+=1
fileName= ("calibration"+str(fileNum)+".npz")



np.savez(fileName, mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs,CAM_ENUM = CAM_ENUM,CAM_WIDTH = CAM_WIDTH,CAM_HEIGHT = CAM_HEIGHT)